In [1]:
from nanorsm_parallel import *

/nsls2/conda/envs/2025-2.2-py312-tiled/lib/python3.12/site-packages/databroker/assets/base_registry.py:14: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


Kafka server not available, Databroker will not be able to upload datum


In [12]:
params = load_json_file()

Loaded data:
{
    "scan ids": [
        346726,
        346728,
        346730,
        346732,
        346734,
        346736
    ],
    "fluorescence data path": "//data//users//2025Q2//Marschilok_2025Q2//xrf//",
    "output path": "//data//users//2025Q2//Marschilok_2025Q2//nanoRSM//346726-346736_2//",
    "element list": [
        "Ni_K",
        "Mn_K",
        "Co_K"
    ],
    "element for alignment": "Co_K",
    "alignment matrix": [
        [
            -0.0,
            -0.0
        ],
        [
            0.5975405753103367,
            1.0138616357968964
        ],
        [
            0.11424652814028846,
            0.16353099796790005
        ],
        [
            -0.15828633640699152,
            0.1321175209316081
        ],
        [
            -0.6232694559850493,
            -0.5702749123030273
        ],
        [
            -1.0081505452115316,
            -1.8318981633881357
        ]
    ],
    "scan dimensions": [
        60,
        60
    ],
    "dete

In [5]:
# load elemental image data, align it with pystackreg and generate a tranform matrix

data_path = params['fluorescence data path']
output_path = params['output path']
if not os.path.exists(output_path):
    os.makedirs(output_path)
    print(f"Created directory: {output_path}")

sid_list = params['scan ids']
if params['alignment matrix'] is None:
    elem = params['element for alignment']
    file_list = [
        f"{data_path}output_tiff_scan2D_{sid}//detsum_{elem}_norm.tiff"
        for sid in sid_list
    ]
    im_stack = load_ims(file_list)
    num_frame,im_row,im_col = np.shape(im_stack)
    im_stack_aligned, trans_matrix = align_im_stack(im_stack) # use pystackreg
    im_stack_test = interp_sub_pix(im_stack,trans_matrix) # verify the alignment is done correctly
    slider_view(im_stack_test)
    params['alignment matrix'] = trans_matrix
    print('if you are happy with the alignment, run the cell again')
else:
    elem_list = params['element list']
    trans_matrix = np.asarray(params['alignment matrix'])
    stack = []
    for i, elem in enumerate(elem_list):
        file_list = [
            f"{data_path}output_tiff_scan2D_{sid}//detsum_{elem}_norm.tiff"
            for sid in sid_list
        ]
        im_stack = load_ims(file_list)
        im_stack_test = interp_sub_pix(im_stack, trans_matrix)
        imp = np.sum(im_stack_test, axis=0)
        stack.append(imp[np.newaxis, ...])
        tifffile.imwrite(f"{output_path}{elem}.tiff",im_stack_test.astype(np.float32),imagej=True)
        print(f"save {elem} to {output_path}{elem}.tiff")
    stack = np.concatenate(stack, axis=0)     

Progress:   0%|          | 0/6 [00:00<?, ?it/s]

Parallel interp_sub_pix:   0%|          | 0/6 [00:00<?, ?it/s]

save Ni_K to //data//users//2025Q2//Marschilok_2025Q2//nanoRSM//346726-346736_2//Ni_K.tiff


Progress:   0%|          | 0/6 [00:00<?, ?it/s]

Parallel interp_sub_pix:   0%|          | 0/6 [00:00<?, ?it/s]

save Mn_K to //data//users//2025Q2//Marschilok_2025Q2//nanoRSM//346726-346736_2//Mn_K.tiff


Progress:   0%|          | 0/6 [00:00<?, ?it/s]

Parallel interp_sub_pix:   0%|          | 0/6 [00:00<?, ?it/s]

save Co_K to //data//users//2025Q2//Marschilok_2025Q2//nanoRSM//346726-346736_2//Co_K.tiff


In [8]:
roi = params['detector roi']
[scan_row, scan_col] = params['scan dimensions']
det = params['detector name']
mon = params['monitor']
diff_data = load_h5_data_db_parallel(sid_list,det=det,mon=mon,roi=roi)
sz = diff_data.shape
diff_data = np.reshape(diff_data,(sz[0],scan_row,scan_col,sz[2],sz[3]))
diff_data = interp_sub_pix(diff_data,trans_matrix)

Progress:   0%|          | 0/6 [00:00<?, ?it/s]

Parallel interp_sub_pix:   0%|          | 0/6 [00:00<?, ?it/s]

In [16]:
### transform to cartesian crystal coordinates (z along hkl and x is the rocking direction)

energy = params['energy']
delta = params['delta']
gamma = params['gamma']
num_angle = params['number of angles']
th_step = params['angle step']
pix = params['pixel size']
det_dist = params['detector distance']
offset = np.asarray(params['roi offset'])
data_store = params['data store'] # this will reduce the data stored. If use 'full' it can be over 100G

method = {'fit_type': 'com',# fitting method: center of mass, 'com' or 'peak'
         'shape': 'gaussian', # peak shape: 'gaussian', 'lorentzian', or 'voigt'
         'n_peaks': [1,1,1]} # number of peaks in each direction, qx, qy, and qz


# generate an object of the RSM class
rsm = RSM(diff_data,energy,delta,gamma,num_angle,th_step,pix,det_dist,offset,stack,elem_list)
# transform from detector coordinates to crystal coordinates
rsm.calcRSM('cryst',data_store,desired_workers=10)
# calculate strain
# 'com', center of mass, is a simple algorithm to calculate the strain. Note: There is an abitrary offset
rsm.calcSTRAIN(method) 
# show results
rsm.disp()
# save results
rsm.save(output_path)
# also save the entire object
save_file = f"{output_path}all_data.obj"
pickle.dump(rsm, open(save_file,'wb'),protocol = 4)

Processing scans in parallel:   0%|          | 0/3600 [00:00<?, ?it/s]

raw det_data is deleted
qxz_data: [pos,qx,qz] with dimensions of (60, 60, 9, 1020)
qyz_data: [pos,qy,qz] with dimensions of (60, 60, 175, 1020)


  0%|          | 0/3600 [00:00<?, ?it/s]

In [ ]:
rsm.run_interactive(scale='linear')

In [ ]:
qz = np.sum(rsm.qxz_data,-2)
plt.figure()
 
x = rsm.zq[0,0,:]
y = qz[35,35,:]
popt, fit_y = fit_peaks(x, y,peak_type='lorentzian',n_peaks=2)
plt.plot(x,y,'black',x,fit_y,'red')
plt.show(block=False)

In [ ]:
method = {'fit_type': 'peak',# fitting method: center of mass, 'com' or 'peak'
         'shape': 'gaussian', # peak shape: 'gaussian', 'lorentzian', or 'voigt'
         'n_peaks': [1,1,1]} # number of peaks in each direction, qx, qy, and qz

rsm.calcSTRAIN(method) 
# show results
rsm.disp()

In [ ]:
rsm.disp()